<a href="https://colab.research.google.com/github/dakilaledesma/FSUS_Explanations/blob/main/notebooks/explanations/CNL_Q600G4F15_FullGrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q drive/MyDrive/UNC/S2022P/GBIF_Q_fourth15.zip -d /content/temp
! mv /content/temp/content/dataset/test /content/test
! mv /content/temp/content/dataset/annotations.txt /content/annotations.txt
! rm -rf /content/temp

CPU times: user 762 ms, sys: 139 ms, total: 902 ms
Wall time: 1min 50s


In [2]:
! pip install timm grad-cam tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 4.1 MB/s 
     |████████████████████████████████| 7.8 MB 22.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.5-py3-none-any.whl size=37027 sha256=9fc567556839964a5c079d483915406bfcec842c1991c5987d6e3fd4cf2be7e0
  Stored in directory: /root/.cache/pip/wheels/f7/c1/fd/59384047f1fab2f99d96b224bdfd0d2b198f896bdb107d66fd
Successfully built grad-cam


In [3]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json
from glob import glob
from tqdm.notebook import tqdm

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

import timm

In [4]:
torch.backends.cudnn.enabled = False

In [5]:
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize((600, 600)),
        transforms.ToTensor(),
        normalize
    ])    

    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    return transf(img).unsqueeze(0)

In [7]:
from collections import defaultdict
import random

ckpt_fn = "drive/MyDrive/UNC/S2022P/output/20220827-184802-convnext_large_384_in22ft1k-600/last.pth.tar"
im_path = "test"
out_path = "resolution/600_cnl"
model_type = "convnext_large_384_in22ft1k"

fl_dict = {}
for idx, fl in enumerate(glob(f"{im_path}/*")):
    bn = os.path.basename(fl)
    fl_dict[bn] = idx
    
images = defaultdict(list)
im_glob = list(glob(f"{im_path}/**/*.*", recursive=True))
random.shuffle(im_glob)

limit = 15 #@param {type:"number"}
for im_fn in im_glob:
    bn = os.path.basename(im_fn)
    cat = int(im_fn.split("/")[-2])
    if len(images[cat]) < limit:
      images[cat].append(im_fn)

images = sum(images.values(), [])
    
print(len(images))

790


In [8]:
model = timm.create_model(
        model_type,
        num_classes=57,
        in_chans=3,
        checkpoint_path=ckpt_fn)

In [9]:
for im_fn in tqdm(images, total=len(list(images))):
    curr_dir = dir()
    cam_list = [["fullgrad", FullGrad(model=model, target_layers=[], use_cuda=True)]]
    
    bn = os.path.basename(im_fn)
    cat = int(im_fn.split("/")[-2])
    
    img = Image.open(im_fn)
    img_t = get_input_tensors(img)
    test_img = np.array(img.resize((600, 600)), dtype='float32')
    test_img /= 255

    targets = [ClassifierOutputTarget(cat)]
    
    for cam_str, cam_method in cam_list:
        if not os.path.exists(f"outs/{out_path}/{cam_str}/{cat}_{bn}"):
            grayscale_cam = cam_method(input_tensor=img_t, targets=targets)
            grayscale_cam = grayscale_cam[0, :]

            visualization = show_cam_on_image(test_img, grayscale_cam, use_rgb=True)
            plt.imshow(visualization)

            try:
                plt.savefig(f"outs/{out_path}/{cam_str}/{cat}_{bn}")
            except FileNotFoundError:
                os.makedirs(f"outs/{out_path}/{cam_str}/")
            finally:
                plt.savefig(f"outs/{out_path}/{cam_str}/{cat}_{bn}")
              
            plt.close()
        
            del grayscale_cam
        
    for name in dir():
        if name not in curr_dir and name != "curr_dir":
            del globals()[name]

  0%|          | 0/790 [00:00<?, ?it/s]

In [10]:
! zip -q -r -0 s2d_outs.zip outs/

In [11]:
! mv s2d_outs.zip /content/drive/MyDrive/UNC/S2022P/q600g4f15_cnl_outs.zip